# 評価方法

分類されていないデータを認識し、どれだけ正しくカテゴリごとに分類できるかを算出した「平均精度」の高さを競い合います。

今回、活用するデータはLSWMD_25519となります。
LSWMD_25519のFailureType項目が分類されていない状態のデータに対し、正しいFailureTypeカテゴリを分類するプログラムを作成し、その平均精度を算出します。
平均精度とは、カテゴリごとに正しく分類できる精度を平均した値です。カテゴリごとに算出した精度（Aが正しく分類された数/Aのデータ数）を足し、カテゴリ数で割ります。

公平な評価を実施するために、以下の制限を設けています。
1. 外部パッケージをインストールするためのセルとsolution関数の中身のみを編集すること
2. 校舎のiMac上で最後のセルの実行時間が15分未満であること　（%%timeitの出力結果を確認してください）

※気になる点がある場合、Discordで気軽にお問合せください。

In [1]:
import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from sklearn.model_selection import train_test_split

外部パッケージを使用する場合、以下の方法でインストールを実施してください。

In [2]:
# 必要な外部パッケージは、以下の内容を編集しインストールしてください
!pip install keras
!pip install opencv-python
!pip install tensorflow-cpu

以下のsolution関数のみ編集してください。

In [3]:
def solution(x_test_df, train_df):
	import numpy as np
	from keras.models import Sequential
	from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
	from keras.callbacks import EarlyStopping
	from sklearn.preprocessing import LabelEncoder
	from tensorflow.keras.utils import to_categorical
	from keras.callbacks import EarlyStopping, ReduceLROnPlateau
	import cv2  # OpenCV をインポート
	from tensorflow.keras.models import Model
	from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Attention
	from sklearn.model_selection import train_test_split
	from sklearn.linear_model import LinearRegression
	from sklearn.neighbors import KNeighborsClassifier
	from sklearn.decomposition import PCA
	from sklearn.preprocessing import StandardScaler

	height, width = 43, 43 # 画像の幅と高さ
	channels = 1
	target_size = (height, width)  # ターゲットサイズを (幅, 高さ) の順で設定
	input_shape = (height, width, channels) # input shape を設定
	train_df, valid_df = train_test_split(train_df, stratify=train_df['failureType'], test_size=0.10, random_state=42)

	def augment_images(images, labels):
		'''
		訓練用のデータの array (3d array) を受け取り, 要素である 2d array を基に拡張された 2d array を 3d array に詰め直して返す関数。
		新たにデータ拡張手法を追加したい場合は、以下の operations に lamda 関数を追加することで可能である。
		'''
		operations = [
			# 実行されるデータ拡張手法
			lambda x: x,
			# lambda x: cv2.rotate(x, cv2.ROTATE_90_CLOCKWISE),
			# lambda x: cv2.rotate(x, cv2.ROTATE_180),
			# lambda x: cv2.rotate(x, cv2.ROTATE_90_COUNTERCLOCKWISE),
			# lambda x: cv2.flip(x, 0),
			# lambda x: cv2.flip(cv2.rotate(x, cv2.ROTATE_90_CLOCKWISE), 0),
			# lambda x: cv2.flip(cv2.rotate(x, cv2.ROTATE_180), 0),
			# lambda x: cv2.flip(cv2.rotate(x, cv2.ROTATE_90_COUNTERCLOCKWISE), 0)
		]

		return np.array([operation(img) for img in images for operation in operations]).reshape(-1, height, width, channels), np.repeat(labels, len(operations), axis=0)

	def cnn_predict(X_train, y_train, X_test, X_valid):

		encoder = LabelEncoder()

		encoded_Y = encoder.fit_transform(y_train)
		y_train = to_categorical(encoded_Y)

		model = Sequential()
		input_layer = Input(shape=input_shape)
		
		# 畳み込み層とプーリング層
		x = Conv2D(32, (5, 5), activation='relu')(input_layer)
		x = MaxPooling2D((2, 2))(x)
		x = BatchNormalization()(x)
		
		x = Conv2D(32, (5, 5), activation='relu')(x)
		x = MaxPooling2D((2, 2))(x)
		x = BatchNormalization()(x)
		
		# Attentionレイヤーの追加
		attention_output = Attention()([x, x])
		
		# Flattenレイヤーの追加
		x = Flatten()(attention_output)
		
		# 全結合層
		x = Dense(128, activation='relu')(x)
		x = Dropout(0.2)(x)
		
		x = Dense(64, activation='relu')(x)
		x = Dropout(0.2)(x)
		
		output_layer = Dense(y_train.shape[1], activation='softmax')(x)
		
		# モデルの定義
		model = Model(inputs=input_layer, outputs=output_layer)
		
		model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
		
		early_stopping = EarlyStopping(monitor='val_loss', patience=20)
		
		# 学習率を自動的に減らすコールバックを作成
		reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.01)
		
		model.fit(X_train, y_train, epochs=1, batch_size=64, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

		X_test = np.array(X_test).reshape(-1, height, width, channels)
		
		y_pred = np.argmax(model.predict(X_test), axis=-1)
		
		first_y_pred = np.argmax(model.predict(X_valid), axis=-1)

		return encoder.inverse_transform(y_pred), encoder.inverse_transform(first_y_pred)

	def knn_predict(X_train, y_train, X_test, X_valid):

		encoder = LabelEncoder()

		encoded_Y = encoder.fit_transform(y_train)
		y_train = to_categorical(encoded_Y)
        
		#PCA次元削減
		pca = PCA(n_components=50)
		X_train_pca = pca.fit_transform(X_train.reshape(X_train.shape[0], -1))
		
		# kNN作成と訓練
		knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=20, metric='minkowski', p=2)
		knn_model.fit(X_train_pca, y_train)
		
		X_test_pca = pca.transform(X_test.reshape(X_test.shape[0], -1))
		
		y_pred = knn_model.predict(X_test_pca)

		X_valid_pca = pca.transform(X_valid.reshape(X_valid.shape[0], -1))
		
		first_y_pred = knn_model.predict(X_valid_pca)
		
		return encoder.inverse_transform(y_pred), encoder.inverse_transform(first_y_pred)

	# メタモデルの作成
	def meta_model_predict(stack_pred, y_valid, y_pred_1, y_pred_2):

		encoder = LabelEncoder()

		encoded_Y = encoder.fit_transform(y_valid)
		y_valid = to_categorical(encoded_Y)
		
		meta_model = LinearRegression()
		meta_model.fit(stack_pred, y_valid)

		stack_test_pred = np.column_stack((y_pred_1, y_pred_2))
		meta_test_pred = meta_model.predict(stack_test_pred)
		return meta_test_pred[0]
		# return encoder.inverse_transform(meta_test_pred[0])
		
	# オリジナルの画像をリサイズ
	resized_images = [cv2.resize(img, target_size[::-1], interpolation=cv2.INTER_LINEAR) for img in train_df['waferMap']]
	valid_resized_images = [cv2.resize(img, target_size[::-1], interpolation=cv2.INTER_LINEAR) for img in valid_df['waferMap']]
	X_test = np.array([cv2.resize(img, target_size[::-1], interpolation=cv2.INTER_LINEAR) for img in x_test_df['waferMap']])
		
	# データ拡張 (ラベルもデータ拡張に合わせて増加)
	X_train, y_train = augment_images(resized_images, np.array(train_df['failureType']))
	X_valid, y_valid = augment_images(valid_resized_images, np.array(valid_df['failureType']))
	del resized_images
	del valid_resized_images

	y_pred_1, first_y_pred_1 = cnn_predict(X_train, y_train, X_test, X_valid)
	y_pred_2, first_y_pred_2 = knn_predict(X_train, y_train, X_test, X_valid)

	encoder = LabelEncoder()

	encoded_Y = encoder.fit_transform(first_y_pred_1)
	first_y_pred_1 = to_categorical(encoded_Y)
	encoded_Y = encoder.fit_transform(first_y_pred_2)
	first_y_pred_2 = to_categorical(encoded_Y)
	encoded_Y = encoder.fit_transform(y_pred_1)
	y_pred_1 = to_categorical(encoded_Y)
	encoded_Y = encoder.fit_transform(y_pred_2)
	y_pred_2 = to_categorical(encoded_Y)

	stack_pred = np.column_stack((first_y_pred_1,first_y_pred_2))

	meta_test_pred = meta_model_predict(stack_pred, y_valid, y_pred_1, y_pred_2)

	return pd.DataFrame({'failureType': encoder.inverse_transform(meta_test_pred)}, index=x_test_df.index)

solution関数は以下のように活用され、平均精度を計算します。

In [4]:
%%timeit -r 1 -n 1

# データのインポート
df = pd.read_pickle("../input/LSWMD_25519.pkl")

# テスト用と学習用のデータを作成（テストする際は、random_stateの値などを編集してみてください）
train_df, test_df = train_test_split(df, stratify=df['failureType'], test_size=0.10, random_state=42)

y_test_df = test_df[['failureType']]
x_test_df = test_df.drop(columns=['failureType'])

# solution関数を実行
user_result_df = solution(x_test_df, train_df)

average_accuracy = 0
# ユーザーの提出物のフォーマット確認
if type(y_test_df) == type(user_result_df) and y_test_df.shape == user_result_df.shape:
    # 平均精度の計算
    accuracies = {}
    for failure_type in df['failureType'].unique():
        y_test_df_by_failure_type = y_test_df[y_test_df['failureType'] == failure_type]
        user_result_df_by_failure_type = user_result_df[y_test_df['failureType'] == failure_type]
        matching_rows = (y_test_df_by_failure_type == user_result_df_by_failure_type).all(axis=1).sum()
        accuracies[failure_type] = matching_rows / len(y_test_df_by_failure_type)
        print(f"不良タイプ {failure_type} の正答率：{accuracies[failure_type] * 100:.2f}%")
    
    average_accuracy = sum(accuracies.values()) / len(accuracies)

print(f"平均精度：{average_accuracy * 100:.2f}%")

import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_test_df, user_result_df, xticks_rotation="vertical")
plt.show()

2023-11-28 08:57:13.852585: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


72/72 [==============================] - 0s 6ms/step


ValueError: y should be a 1d array, got an array of shape (2552, 8) instead.